# Root Mean Square Error Calculator for Different Driveability Back Analysis Cases

In [ ]:
# Importing required tools
import pandas as pd 
import numpy as np
import xlrd
import os

In [ ]:
# Importing local created functions from large_runfunc.py
from large_runfunc import runfunc_ID, runfunc_BA, runfunc_BE

In [ ]:
# Importing local created functions from eval_R_MSE.py
from eval_R_MSE import eval_error_BA, eval_error_BE

### State how many cases have been processed

In [ ]:
#How many cases?
n = input("How many cases have been processed?:")

In [ ]:
#Creating the case folders
Ix = ['Case' + str(num) for num in range(0,(int(n)+1))]

for f in Ix:
    if f==0:
        continue
    else:
        newpath = os.getcwd() + '\\' + f
        #Creating the folder if it does not exist already
        if not os.path.exists(newpath):
            os.makedirs(newpath)

### Have the Back-Analysis GRLWEAP OPILE Plotting files been placed in the Case folders?

In [ ]:
#Files placed?

# Giving 5 tries at typing it correctly
for i in range(0,6):
    In = input("Have you placed all the required Back Analysis Case GRLWEAP_OPILE_Plotting.xlsm files in each Case folder? (y/n)")
    if In == "y":
        print("Nice 1.")
        break
    elif In == "n":
        raise NameError("Please place the required Back Analysis Case GRLWEAP_OPILE_Plotting.xlsm files in each Case folder and then Restart & Run kernel again.")
    else:
        print("Please answer y or n")
    

In [ ]:
# Make sure that all of the BA sheets have the same name and no of sheets in the files!

# Getting list of the sheets in the worbook 
xls = xlrd.open_workbook(r'Case1\\GRLWEAP_OPILE_Plotting.xlsm', on_demand=True)
sheets = xls.sheet_names()

# Removing any sheets that are plots, summary or base and keeping ones with data to be used
sheets = [names for names in sheets if "Summary" not in names]
sheets = [names for names in sheets if "Base" not in names]
sheets = [names for names in sheets if "Plots" not in names]


#Creating the final storage location for all the cases results (New columns added later)
results = pd.DataFrame(columns = ['AVG_Location_RMSE'], index = Ix)

### Does the Best Estimate Line change between Cases?

In [ ]:
# For Best Estimate (BE)
#Insert BE result into the results frame. 

# Creating the list to store the prepped data
ID_list = []
BE_list =[]

# Creating the list to store the location RMSE vals
RMSE_BE_list = []


#Files placed?

# Giving 5 tries at typing it correctly
for i in range(0,6):
    BE_ch = input("Does the Best Estimate line change between cases? (y/n)")
    # IF BE changes, find average across all the Cases of BE
    if BE_ch == "y":

       
        # iterating through the cases
        for case in Ix:
            # Skipping BE case (Case0)
            if case == 'Case0':
                continue 

            # Creating the list to store the prepped data
            ID_list = []
            BE_list = []


            # opening the case plotting file and iterating through locations 
            for sheet in sheets:
                fileloc =  case + '\\GRLWEAP_OPILE_Plotting.xlsm'
                df1 = pd.read_excel(fileloc, sheet_name = sheet)


                #Calling the prep functions
                fullID = runfunc_ID(df1)
                fullBE = runfunc_BE(df1)

                #Appending the prepped data to the list
                ID_list.append(fullID)
                BE_list.append(fullBE)


                # Taking the original columns from fullID and fullBE
                x1_ID = fullID['SRD']
                y1_ID = fullID['depth']

                # Smoothing the ID by applying a rolling mean
                x1mean = x1_ID.rolling(50, min_periods=1).mean()
                y1mean = y1_ID


                # Defining the smoothed frame for ID to be used for eval MSE
                ID_0 = pd.DataFrame(columns = ['depth','SRD'])
                ID_0['SRD'] = x1mean
                ID_0['depth'] = y1mean

                # Defining the BE frame to use
                BE_0 = fullBE
                
                #Calling the RMSE function for the BE
                MSE_BE, RMSE_BE = eval_error_BE(BE_0,ID_0)

            #     print(MSE_BE)
            #     print(RMSE_BE)

                #Appending the RMSE for BE
                RMSE_BE_list.append(RMSE_BE)

                print(RMSE_BE_list)
            del df1, fullID, fullBE, x1_ID, y1_ID, x1mean, y1mean, ID_0, BE_0
        
        #Appending the AVG RMSE for BE at end of case loop
        results.iloc[0,0] = np.mean(RMSE_BE_list)  
        
        break
        
    # IF BE does not change use the GRLWEAP OPILE PLOTTING sheet from Case0    
    elif BE_ch == "n":
        # Looping through the sheets
        for sheet in sheets:
            df1 = pd.read_excel(r'Case0\\GRLWEAP_OPILE_Plotting.xlsm', sheet_name = sheet)

           #Calling the prep functions
            fullID = runfunc_ID(df1)
            fullBE = runfunc_BE(df1)

            #Appending the prepped data to the list
            ID_list.append(fullID)
            BE_list.append(fullBE)


            # Taking the original columns from fullID and fullBE
            x1_ID = fullID['SRD']
            y1_ID = fullID['depth']

            # Smoothing the ID by applying a rolling mean
            x1mean = x1_ID.rolling(50, min_periods=1).mean()
            y1mean = y1_ID



            # Defining the smoothed frame for ID to be used for eval MSE
            ID_0 = pd.DataFrame(columns = ['depth','SRD'])
            ID_0['SRD'] = x1mean
            ID_0['depth'] = y1mean

            # Defining the BE frame to use
            BE_0 = fullBE

            #Calling the RMSE function for the BE
            MSE_BE, RMSE_BE = eval_error_BE(BE_0,ID_0)

        #     print(MSE_BE)
        #     print(RMSE_BE)
            RMSE_BE_list.append(RMSE_BE)

        #Appending the AVG RMSE for BE at end of loop
        results.iloc[0,0] = np.mean(RMSE_BE_list) 
        break
    
    else:
        print("Please answer y or n")



In [ ]:
# For Backanalysis
# Defining the case number
k = 1

# iterating through the cases
for case in Ix:
    # Skipping BE case (Case0)
    if case == 'Case0':
        continue 
        
    # Creating the list to store the prepped data
    ID_list = []
    BA_list = []

    # Creating the list to store the location RMSE vals
    RMSE_BA_list = []
    
    # opening the case plotting file and iterating through locations 
    for sheet in sheets:
        fileloc =  case + '\\GRLWEAP_OPILE_Plotting.xlsm'
        df1 = pd.read_excel(fileloc, sheet_name = sheet)


        #Calling the prep functions
        fullID = runfunc_ID(df1)
        fullBA = runfunc_BA(df1)

        #Appending the prepped data to the list
        ID_list.append(fullID)
        BA_list.append(fullBA)


        # Taking the original columns from fullID and fullBA
        x1_ID = fullID['SRD']
        y1_ID = fullID['depth']


        # Smoothing the ID by applying a rolling mean
        x1mean = x1_ID.rolling(50, min_periods=1).mean()
        y1mean = y1_ID



        # Defining the smoothed frame for ID to be used for eval MSE
        ID_0 = pd.DataFrame(columns = ['depth','SRD'])
        ID_0['SRD'] = x1mean
        ID_0['depth'] = y1mean

        # Defining the BA frame to use
        BA_0 = fullBA

        #Calling the RMSE function for the BA
        MSE_BA, RMSE_BA = eval_error_BA(BA_0,ID_0)

    #     print(MSE_BA)
    #     print(RMSE_BA)

        #Appending the RMSE for BA
        RMSE_BA_list.append(RMSE_BA)

    #Appending the AVG RMSE for BA at end of sheet loop
    results.iloc[k,0] = np.mean(RMSE_BA_list) 
    del df1, fullID, fullBA, x1_ID, y1_ID, x1mean, y1mean, ID_0, BA_0
    k+=1

In [ ]:
# Changing the first column to indicate that it is for BE
results = results.rename(index={'Case0': 'BE'})

In [ ]:
print("These are your results per case:")
results

In [ ]:
#Saving the results to the system
results.to_excel('Case_Results.xlsx')

### The Results have been saved to your local system as Case_Results excel file.